In [1]:
import sqlite3
import pandas as pd
from haversine import haversine
import itertools
import time
start = time.time()

In [2]:
north_pole = (90,0)
weight_limit = 1000.0

for i in range(90,-90,int(-180/1.26)): 
        print ("i=: "+str(i))

for i in range(180,-180,int(-360/1.4)): 
        print ("i=: "+str(i))

i=: 90
i=: -52
i=: 180
i=: -77


In [3]:
def bb_sort(ll):
    s_limit = 500
    optimal = False
    ll = [[0,north_pole,10]] + ll[:] + [[0,north_pole,10]] 
    while not optimal:
        optimal = True
        for i in range(1,len(ll) - 4):
            lcopy=[[] for _ in range(24)]   
            b=list(itertools.permutations(([0,1,2,3])))
            for k in range(2,24):
                lcopy[k]=ll[:]
                lcopy[k][i], lcopy[k][i+1], lcopy[k][i+2],lcopy[k][i+3] = ll[i+b[k][0]][:], ll[i+b[k][1]][:],ll[i+b[k][2]][:],ll[i+b[k][3]][:]

            minDist=path_opt_test(ll[1:-1])
            kmn=0
            for k in range(2,24):
                tmp=path_opt_test(lcopy[k][1:-1])
                if tmp<minDist:
                    kmn=k
                    minDist=tmp
            if kmn>0:
            
                ll = lcopy[kmn][:]
                optimal = False
                s_limit -= 1
                if s_limit < 0:
                    optimal = True
                    break
                continue
        
    return ll[1:-1]

In [5]:
def path_opt_test(llo):
    f_ = 0.0
    d_ = 0.0
    l_ = north_pole
    for i in range(len(llo)):
        d_ += haversine(l_, llo[i][1])
        f_ += d_ * llo[i][2]
        l_ = llo[i][1]
    d_ += haversine(l_, north_pole)
    f_ += d_ * 10 
    return f_

gifts = pd.read_csv("D:\santas-stolen-sleigh/gifts.csv").fillna(" ")
c = sqlite3.connect(":memory:")
gifts.to_sql("gifts",c)

100000

In [6]:
cu = c.cursor()
cu.execute("ALTER TABLE gifts ADD COLUMN 'TripId' INT;")
cu.execute("ALTER TABLE gifts ADD COLUMN 'i' INT;")
cu.execute("ALTER TABLE gifts ADD COLUMN 'j' INT;")
c.commit()


In [7]:
for n in [1.26]: 
    i_ = 0
    j_ = 0
    for i in range(90,-90,int(-180/n)): 
        i_ += 1
        j_ = 0
        for j in range(180,-180,int(-360/(n+0.0))): 
            j_ += 1
            cu = c.cursor()
            cu.execute("UPDATE gifts SET i=" + str(i_) + ", j=" + str(j_) + " WHERE ((Latitude BETWEEN " + 
            str(i - (180/n)) + " AND  " + str(i) + ") AND (Longitude BETWEEN " + str(j - (360/n)) + 
            " AND  " + str(j) + "));")
            c.commit()
    
    for limit_ in [67]: 
        trips = pd.read_sql("SELECT * FROM (SELECT * FROM gifts WHERE TripId IS NULL ORDER BY i, j, Longitude, Latitude LIMIT " + 
        str(limit_) + " ) ORDER BY Latitude DESC",c)
        t_ = 0
        while len(trips.GiftId)>0:
            g = []
            t_ += 1
            w_ = 0.0
            for i in range(len(trips.GiftId)):
                if (w_ + float(trips.Weight[i]))<= weight_limit:
                    w_ += float(trips.Weight[i])
                    g.append(trips.GiftId[i])
            cu = c.cursor()
            cu.execute("UPDATE gifts SET TripId = " + str(t_) + " WHERE GiftId IN(" + (",").join(map(str,g)) + ");")
            c.commit()
        
            trips = pd.read_sql("SELECT * FROM (SELECT * FROM gifts WHERE TripId IS NULL ORDER BY i, j, Longitude, Latitude LIMIT " + 
            str(limit_) + " ) ORDER BY Latitude DESC",c)
            
        
        ou_ = open("submission_opt" + str(limit_) + " " + str(n) + ".csv","w")
        ou_.write("TripId,GiftId\n")
        bm = 0.0
        submission = pd.read_sql("SELECT TripId FROM gifts GROUP BY TripId ORDER BY TripId;", c)
        for s_ in range(len(submission.TripId)):
            trip = pd.read_sql("SELECT GiftId, Latitude, Longitude, Weight FROM gifts WHERE TripId = " +
            str(submission.TripId[s_]) + " ORDER BY Latitude DESC, Longitude ASC;",c)
            a = []
            for x_ in range(len(trip.GiftId)):
                a.append([trip.GiftId[x_],(trip.Latitude[x_],trip.Longitude[x_]),trip.Weight[x_]])
            b = bb_sort(a)
            if path_opt_test(a) <= path_opt_test(b):
                print(submission.TripId[s_], "No Change", path_opt_test(a) )
                bm += path_opt_test(a)
                for y_ in range(len(a)):
                    ou_.write(str(submission.TripId[s_])+","+str(a[y_][0])+"\n")
            else:
                print(submission.TripId[s_], "Optimized", int(1000*path_opt_test(b)/path_opt_test(a))/10.0,"%")
                bm += path_opt_test(b)
                for y_ in range(len(b)):
                    ou_.write(str(submission.TripId[s_])+","+str(b[y_][0])+"\n")
        ou_.close()
        
        print ("Submission score: ")
        print(n, limit_, bm)
       
        benchmark = 12507006788.2+1
        if bm < benchmark:
            print(n, limit_, "Improvement", bm, bm - benchmark, benchmark)
        else:
            print(n, limit_, "Try again", bm, bm - benchmark, benchmark)
        cu = c.cursor()
        cu.execute("UPDATE gifts SET TripId = NULL;")
        c.commit()

1 No Change 5184406.731892624
2 No Change 5468360.371226408
3 No Change 4760038.409272012
4 No Change 5185791.022662973
5 No Change 4342404.437806045
6 Optimized 99.9 %
7 Optimized 99.9 %
8 No Change 5735319.277863557
9 No Change 4925104.002846945
10 Optimized 99.9 %
11 No Change 5729746.471974414
12 No Change 5821196.7624971
13 Optimized 99.9 %
14 Optimized 99.9 %
15 No Change 4961183.513514459
16 No Change 5587953.445966236
17 No Change 5487218.945106384
18 No Change 5077512.697789635
19 No Change 4402863.725801591
20 Optimized 99.9 %
21 No Change 5188850.511134735
22 No Change 4861247.555466584
23 Optimized 99.9 %
24 No Change 5173156.914153158
25 Optimized 99.9 %
26 No Change 5678567.796780514
27 No Change 5684879.773071027
28 No Change 5553817.708858512
29 No Change 5528586.61817018
30 No Change 5625583.626143171
31 Optimized 99.9 %
32 No Change 5499939.288672341
33 No Change 5680301.243962338
34 No Change 5085748.941133572
35 Optimized 99.9 %
36 Optimized 99.9 %
37 No Change 4522

293 Optimized 99.9 %
294 Optimized 99.9 %
295 Optimized 99.9 %
296 Optimized 99.9 %
297 No Change 8211130.199050839
298 No Change 9534373.251054393
299 No Change 7927119.184616337
300 No Change 7000713.296996834
301 Optimized 99.9 %
302 No Change 8480856.958053743
303 Optimized 99.9 %
304 No Change 9409892.970118316
305 Optimized 99.9 %
306 Optimized 99.9 %
307 Optimized 99.9 %
308 Optimized 99.9 %
309 Optimized 99.9 %
310 Optimized 99.9 %
311 Optimized 99.9 %
312 Optimized 99.9 %
313 Optimized 99.9 %
314 Optimized 99.9 %
315 Optimized 99.9 %
316 Optimized 99.8 %
317 Optimized 99.7 %
318 Optimized 99.9 %
319 Optimized 99.7 %
320 Optimized 99.8 %
321 Optimized 99.9 %
322 Optimized 99.5 %
323 Optimized 99.4 %
324 Optimized 99.1 %
325 Optimized 99.1 %
326 Optimized 99.5 %
327 Optimized 99.6 %
328 Optimized 99.1 %
329 Optimized 99.0 %
330 Optimized 97.7 %
331 Optimized 98.1 %
332 Optimized 95.5 %
333 Optimized 96.1 %
334 Optimized 99.2 %
335 Optimized 99.5 %
336 Optimized 99.0 %
337 Optimi

588 No Change 8282216.196594581
589 No Change 8890991.913845958
590 Optimized 99.9 %
591 No Change 7612224.701683579
592 No Change 7489321.278962351
593 No Change 9575406.56532562
594 No Change 9641049.368588181
595 No Change 8628027.896376386
596 No Change 9354775.805576377
597 No Change 9372913.96894702
598 No Change 8233264.571782145
599 No Change 9099086.037392626
600 No Change 9289911.671710363
601 No Change 7400390.743722253
602 No Change 7599984.614357783
603 Optimized 99.9 %
604 No Change 6919647.07480386
605 No Change 8001163.125168076
606 No Change 8825240.16768036
607 No Change 8174545.975681171
608 No Change 9047086.380721634
609 No Change 8309895.4650235325
610 No Change 9357523.843183074
611 No Change 8076248.334368649
612 No Change 8622626.46680187
613 No Change 8302977.812279452
614 No Change 8605731.958398579
615 No Change 7667484.750122328
616 No Change 7765583.945757195
617 No Change 8393934.662963726
618 No Change 8828715.637404837
619 No Change 9379584.146585608
62

875 No Change 5759196.484592539
876 No Change 5463505.5725509925
877 No Change 6013978.559585542
878 Optimized 99.9 %
879 Optimized 99.8 %
880 No Change 5959888.681247387
881 No Change 5556867.280922785
882 Optimized 99.9 %
883 Optimized 99.9 %
884 No Change 5012535.251027056
885 Optimized 99.9 %
886 Optimized 99.9 %
887 No Change 5205065.745557089
888 No Change 5170853.859749028
889 No Change 5188013.4200006295
890 Optimized 99.9 %
891 Optimized 99.9 %
892 Optimized 99.9 %
893 No Change 6408056.671303151
894 No Change 4903125.997204802
895 Optimized 99.9 %
896 No Change 5340076.723132197
897 No Change 6340029.665100145
898 No Change 6296559.788349985
899 No Change 5673414.71209107
900 Optimized 99.9 %
901 No Change 5960286.103260333
902 Optimized 99.9 %
903 No Change 5735420.303347871
904 Optimized 99.9 %
905 Optimized 99.9 %
906 Optimized 99.9 %
907 Optimized 99.9 %
908 No Change 5241977.911055708
909 No Change 5283572.163895994
910 No Change 5608557.009791849
911 No Change 5082818.3

1157 No Change 9752467.93144546
1158 Optimized 99.9 %
1159 No Change 6852869.420224956
1160 No Change 7852966.918418396
1161 No Change 10192186.008155514
1162 No Change 8805251.048048383
1163 No Change 7796815.63363265
1164 No Change 7699433.873940658
1165 No Change 7376724.45738478
1166 Optimized 99.9 %
1167 No Change 5059206.97137515
1168 Optimized 99.9 %
1169 No Change 9393440.908195622
1170 Optimized 99.9 %
1171 Optimized 99.9 %
1172 Optimized 99.9 %
1173 No Change 7707122.468149063
1174 No Change 7655165.827373821
1175 Optimized 99.9 %
1176 No Change 8798784.071776703
1177 No Change 9422485.37061738
1178 No Change 8915960.322859991
1179 Optimized 99.9 %
1180 No Change 7657243.70984708
1181 No Change 7897376.167166672
1182 No Change 9094703.409994986
1183 Optimized 99.9 %
1184 Optimized 99.9 %
1185 Optimized 99.9 %
1186 Optimized 99.9 %
1187 Optimized 99.9 %
1188 Optimized 99.9 %
1189 Optimized 99.9 %
1190 No Change 8727172.89823395
1191 Optimized 99.9 %
1192 Optimized 99.9 %
1193 

1492 Optimized 99.9 %
1493 Optimized 99.9 %
1494 Optimized 99.8 %
1495 Optimized 99.3 %
1496 Optimized 99.7 %
1497 Optimized 99.5 %
1498 Optimized 99.3 %
1499 Optimized 99.5 %
1500 Optimized 99.3 %
1501 Optimized 99.6 %
1502 Optimized 99.2 %
1503 Optimized 99.0 %
1504 Optimized 99.5 %
1505 Optimized 99.0 %
1506 Optimized 99.8 %
1507 Optimized 99.5 %
1508 Optimized 99.6 %
1509 Optimized 99.8 %
1510 Optimized 99.7 %
1511 Optimized 99.8 %
1512 Optimized 99.8 %
1513 Optimized 99.7 %
1514 Optimized 99.2 %
1515 Optimized 99.7 %
1516 Optimized 99.7 %
1517 Optimized 99.6 %
1518 Optimized 99.8 %
1519 Optimized 99.5 %
1520 Optimized 99.9 %
1521 Optimized 99.8 %
1522 Optimized 99.4 %
1523 Optimized 99.9 %
Submission score: 
1.26 67 12507024063.576614
1.26 67 Try again 12507024063.576614 17274.376613616943 12507006789.2


In [8]:
end = time.time()
elapsed = end - start
print("Elapsed time:",elapsed)


Elapsed time: 492.8390018939972
